In [1]:
import sys 

class Edge:
    def __init__(self, b, e, v):
        self.val = v 
        self.begin = b
        self.end = e 
    def exchange(self, edge):
        #交换两个Pair对象
        v = self.val
        b = self.begin
        e = self.end
        
        self.val = edge.val
        self.begin = edge.begin
        self.end = edge.end 
        
        edge.val = v 
        edge.begin = b
        edge.end = e
    def getEdge(self):
        return (self.begin, self.end)

In [3]:
class HeapEdgeSort:
    def __init__(self, array):
        self.heapSize = len(array)
        self.heapArray = array
        #增加一个dict,用于记录边在数组中的下标
        self.edgePosDict = {}
        for i in range(len(array)):
            self.edgePosDict[array[i].getEdge()] = i
        
    def parent(self, i):
        #获得父亲节点在数组中的下标
        return int(i/2)
    def left(self, i):
        #获得左孩子在数组中下标
        return 2*i
    def right(self, i):
        #获得右孩子在数组中下标
        return 2*i+1
    def maxHeapify(self, i):
        '''
        把下标为i的节点与孩子节点进行置换,先找出左右孩子节点中最大值，然后将当前节点与之互换，
        接着进入置换后的节点，继续执行置换流程，直到底部从而维持二叉树符合堆的性质
        '''
        #先把坐标i加一，因为数组下标从0开始，但是算法中元素的下标从1开始
        i += 1
        l = self.left(i)
        r = self.right(i)
        #把下标都减一，因为数组下标从0开始，算法中元素的下标从1开始
        i -= 1
        l -= 1
        r -= 1
        
        #从左右孩子节点中找出最大那个
        largest = -1
        if l < self.heapSize and self.heapArray[l].val > self.heapArray[i].val:
            largest = l
        else:
            largest = i 
        if r < self.heapSize and self.heapArray[r].val > self.heapArray[largest].val:
            largest = r 
        
        #如果左右孩子节点比父节点大，那么将父节点与对应的孩子节点置换
        if largest != i:
           self.heapArray[largest].exchange(self.heapArray[i])
           #在dict中记录edge在数组中的位置
           self.edgePosDict[self.heapArray[largest].getEdge()] = largest 
           self.edgePosDict[self.heapArray[i].getEdge()] = i 
           
           #置换后进入下一层继续执行置换流程
           self.maxHeapify(largest)
           
    def buildMaxHeap(self):
        #构建大堆
        '''
        如果元素在数组中的下标是i，那么左孩子下标为2*i,右孩子为2*i+1
        于是所有处于出自后半部的元素只能是叶子节点，注意到单个节点本身就能构成大堆，所以叶子节点本身就满足大堆的性质
        '''
        i = int(self.heapSize / 2)
        while i >= 0:
            self.maxHeapify(i)
            i -= 1
        return self.heapArray
    
    def maxmun(self): 
        return self.heapArray[0]
    
    def extractMaximun(self): 
        if self.heapSize < 1:
            return None
        
        max = self.heapArray[0]
        #将最后一个元素的值设置成根节点的值
        self.heapArray[0] = self.heapArray[self.heapSize - 1]
        self.edgePosDict[self.heapArray[0].getEdge()] = 0
        
        self.heapSize -= 1
        self.heapArray.pop()
        #调用maxHeapify将前n-1个元素调整成大堆结构
        self.maxHeapify(0)
        
        return max
    def increaseKey(self, i, k):
        #改变下标为i的节点值
        if self.heapArray[i].val >= k:
            return
        self.heapArray[i].val = k
        #元素值增大后，它要与父节点置换以便满足大堆性质
        while i > 0 and self.heapArray[self.parent(i)].val < self.heapArray[i].val:
            self.heapArray[self.parent(i)].exchange(self.heapArray[i])
            #记录下edge在数组中的位置
            self.edgePosDict[self.heapArray[self.parent(i)].getEdge()] = self.parent(i)
            self.edgePosDict[self.heapArray[i].getEdge()] = i 
            
            i = self.parent(i)
    def insert(self, edge):
        #在数组末尾添加一个最小值
        p = Edge(-sys.maxsize, edge.begin, edge.end)
        self.heapArray.append(p)
        #然后调用increaseKey将它增加到val
        self.heapSize += 1
        self.increaseKey(self.heapSize - 1, edge.val)
        return self.heapArray
    def increaseEdge(self, edge, v):
        #改变指定边的值
        i = self.edgePosDict.get(edge.getEdge(), None)
        if i is not None:
            self.increaseKey(i, v)
    def getEdgeFromHeap(self, edge):
        pos =  self.edgePosDict[edge]
        return self.heapArray[pos]
    def isEmpty(self):
        return len(self.heapArray) == 0


In [6]:
class MinimunSpanningTree:
    def __init__(self, vCount, start, edges):
        #图中的总节点数
        self.vCount = vCount
        #起始节点
        self.start = start 
        self.edgeList = []
        self.startEdges = {}
        self.allShortestPath = {}
        self.edgesVisited = {}
        self.edgeAndWeightMap = {}
        #初始化从起点到其他每一个节点的边,节点编号从1开始，所以遍历所有节点时用vCount+1
        for i in range(0, vCount+1):
            if i != start and i != 0:
                #这里的边要取负数，因为算法中庸的是小堆，我们用的是大堆
                e = Edge(start, i, -sys.maxsize)
                self.startEdges[e.getEdge()] = e 
            self.edgeList.append([])
            
        '''
        我们对图中的边是这么组织的，假设以节点1出发的边是((1,2), 4), ((1,4),6), ((1,7),9)
        也就是从节点1出发有三条边，分别是从节点1进入节点2，距离是4；节点1进入节点4，距离是6；节点1进入节点7距离是9
        那么我们在代码中的表示是：
        self.edgeList[1] = [((1,2),4), ((1,4),6), ((1,7),9)]
        如此当我们想查找所有从节点1出发的边，只要访问self.edgeList[1]即可
        ''' 
        #每一条边的格式是[(begin,end), distance]
        for edge in edges:
            #把每条边和它的权重对应起来
            self.edgeAndWeightMap[edge[0]] = edge[1]
            
            self.edgeList[edge[0][0]].append(edge)
            #如果当前边的起点是start，那么修改startEdge里面边的值 
            if edge[0][0] == start:
                #我们用的是大堆，为了实现小堆的效果，需要把边的值变成负数
                self.startEdges[edge[0]].val = -edge[1]
                
        self.edgeHeap = HeapEdgeSort(list(self.startEdges.values()))
        self.edgeHeap.buildMaxHeap()
        
        self.getAllShortestPath()
    
    def getAllShortestPath(self):
        shortestPath = self.edgeHeap.extractMaximun() 
        edge = shortestPath.getEdge()
        self.allShortestPath[edge[1]] = edge[0] 
        
        while self.edgeHeap.isEmpty() is False:
            '''
            从堆中取出由起始节点到其他节点距离最小那条边（start, u)，然后遍历由节点u出发的所有边，例如遍历到(u,v)时
            判断从start 到 u 再到v的距离是否比当前start到v的距离更近，如果是，那么更新start到v的距离
            '''
            edgeFromStartToU = shortestPath
        
            #记住我们存在堆栈中边的距离是负数
            distanceFromStartToU = abs(edgeFromStartToU.val)
            
            v = shortestPath.end 
            #遍历所有从v出发的边
            for edge in self.edgeList[v]:
                 #记住我们存在堆栈中边的距离是负数
                 distanceFromUToV = edge[1]
                 #因为是无向图，要防止访问同一条便，例如边(1,4),得到节点4，接着遍历节点4出发的所有边，其中便(4,1)与边(1,4)是一样的，遍历时要忽略掉
                 if self.start == edge[0][1]:
                     continue
                 #判断当前边是否被访问过
                 if self.edgesVisited.get(edge[0], None) is not None:
                     continue 
                 
                 #把当前边标记为已经访问，注意当我们访问了（u,v)那意味着(v,u)也已经被访问了，因为是无向图
                 self.edgesVisited[(edge[0][0], edge[0][1])] = 1
                 self.edgesVisited[(edge[0][1], edge[0][0])] = 1
                 
                 edgeFromStartToV = self.edgeHeap.getEdgeFromHeap((self.start, edge[0][1])) 
                 #获得从起始节点到节点v的距离
                 distanceFromStartToV = abs(edgeFromStartToV.val)
                 #选取横跨两个点集的最小权重边
                 if distanceFromStartToV >  distanceFromUToV:
                     #记录每个点所对应的最短边，如果当前最短边是(u,v)，那么以v为k，u为value存储在表中
                     self.allShortestPath[edge[0][1]] = edge[0][0]
                                          
                     self.edgeHeap.increaseEdge(edgeFromStartToV, -distanceFromUToV) 
                     
            shortestPath = self.edgeHeap.extractMaximun()
            
    def showAllShortestPath(self):
        for v in self.allShortestPath.keys():
           u = self.allShortestPath[v]
           #将当前访问到的边记录下来
           print("add edge {0} - {1} with weight {2}".format(u, v, self.edgeAndWeightMap[(u,v)]))


In [8]:
vCount = 6
start = 1
edges = [((1,2),5),((2,1),5), ((1,3),6),((3,1),6), ((1,4),4),((4,1),4), 
         ((2,3),1),((3,2),1), ((2,4),2),((4,2),2), 
         ((3,4),2),((4,3),2), ((3,5),5),((5,3),5), ((3,6),3),((6,3),3), 
         ((5,6),4),((6,5),4), ((4,6),4),((6,4),4)] 
ds = MinimunSpanningTree(vCount, start, edges)    
ds.showAllShortestPath()          

add edge 1 - 4 with weight 4
add edge 4 - 2 with weight 2
add edge 2 - 3 with weight 1
add edge 3 - 6 with weight 3
add edge 6 - 5 with weight 4


In [9]:
class Pair:
    def __init__(self, v, b, e):
        self.val = v 
        self.begin = b
        self.end = e 
    def exchange(self, pair):
        #交换两个Pair对象
        v = self.val
        b = self.begin
        e = self.end
        
        self.val = pair.val
        self.begin = pair.begin
        self.end = pair.end 
        
        pair.val = v 
        pair.begin = b
        pair.end = e  

In [10]:
class HeapPairSort:
    def __init__(self, array):
        self.heapSize = len(array)
        self.heapArray = array
        
    def parent(self, i):
        #获得父亲节点在数组中的下标
        return int(i/2)
    def left(self, i):
        #获得左孩子在数组中下标
        return 2*i
    def right(self, i):
        #获得右孩子在数组中下标
        return 2*i+1
    def maxHeapify(self, i):
        '''
        把下标为i的节点与孩子节点进行置换,先找出左右孩子节点中最大值，然后将当前节点与之互换，
        接着进入置换后的节点，继续执行置换流程，直到底部从而维持二叉树符合堆的性质
        '''
        #先把坐标i加一，因为数组下标从0开始，但是算法中元素的下标从1开始
        i += 1
        l = self.left(i)
        r = self.right(i)
        #把下标都减一，因为数组下标从0开始，算法中元素的下标从1开始
        i -= 1
        l -= 1
        r -= 1
        
        #从左右孩子节点中找出最大那个
        largest = -1
        if l < self.heapSize and self.heapArray[l].val > self.heapArray[i].val:
            largest = l
        else:
            largest = i 
        if r < self.heapSize and self.heapArray[r].val > self.heapArray[largest].val:
            largest = r 
        
        #如果左右孩子节点比父节点大，那么将父节点与对应的孩子节点置换
        if largest != i:
           self.heapArray[largest].exchange(self.heapArray[i])
           #置换后进入下一层继续执行置换流程
           self.maxHeapify(largest)
           
    def buildMaxHeap(self):
        #构建大堆
        '''
        如果元素在数组中的下标是i，那么左孩子下标为2*i,右孩子为2*i+1
        于是所有处于出自后半部的元素只能是叶子节点，注意到单个节点本身就能构成大堆，所以叶子节点本身就满足大堆的性质
        '''
        i = int(self.heapSize / 2)
        while i >= 0:
            self.maxHeapify(i)
            i -= 1
        return self.heapArray
    
    def maxmun(self): 
        return self.heapArray[0]
    
    def extractMaximun(self): 
        if self.heapSize < 1:
            return None
        
        max = self.heapArray[0]
        #将最后一个元素的值设置成根节点的值
        self.heapArray[0] = self.heapArray[self.heapSize - 1]
        self.heapSize -= 1
        self.heapArray.pop()
        #调用maxHeapify将前n-1个元素调整成大堆结构
        self.maxHeapify(0)
        
        return max
    def increaseKey(self, i, k):
        #改变下标为i的节点值
        if self.heapArray[i].val >= k:
            return
        self.heapArray[i].val = k
        #元素值增大后，它要与父节点置换以便满足大堆性质
        while i > 0 and self.heapArray[self.parent(i)].val < self.heapArray[i].val:
            self.heapArray[self.parent(i)].exchange(self.heapArray[i])
            i = self.parent(i)
    def insert(self, pair):
        #在数组末尾添加一个最小值
        p = Pair(-sys.maxsize, pair.begin, pair.end)
        self.heapArray.append(p)
        #然后调用increaseKey将它增加到val
        self.heapSize += 1
        self.increaseKey(self.heapSize - 1, pair.val)
        return self.heapArray


In [11]:
class HoffManCoding:
    def  __init__(self, f):
        if len(f) < 2:
            raise RuntimeError("frequency array should contain at least two elements")
        #我们原来实现的是大堆排序，为了能获得数组中的最小值，我们把数组元素转变为负数
        self.frequency = []
        self.treeRoot = None 
        self.encodeString= []
        for i in f:
            n = Pair(-i, None, None)
            self.frequency.append(n)
        self.heapSort = HeapPairSort(self.frequency)
        self.heapSort.buildMaxHeap()
        self.encode()
        
    def encode(self):
        node = None 
        while self.heapSort.heapSize > 1:
            #获取频率最小的两个节点用于构造二叉树
            left = self.heapSort.extractMaximun()
            right = self.heapSort.extractMaximun()
            
            node = Pair(left.val + right.val, left, right)
            self.heapSort.insert(node)
            
        self.treeRoot = node 
    def printEncodings(self):
        self.printEncodingString(self.treeRoot)
        
    def printEncodingString(self, node):
        #中序遍历二叉树，遇到叶子节点时打印编码字符串
        if node is None:
            return 
        if node.begin is None and node.end is None:
            print("symbol with frequency {0} has encoding string {1}".format(abs(node.val), str(self.encodeString)))
            return 
        if node.begin is not None:
            #进入左节点时，对应编码字符0
            self.encodeString.append(0)
            self.printEncodingString(node.begin)
            self.encodeString.pop()
        if node.end is not None:
            #进入右节点时，对应编码字符1
            self.encodeString.append(1)
            self.printEncodingString(node.end)
            self.encodeString.pop()
        
        return


In [12]:
f = [70, 3, 20, 37]
hc = HoffManCoding(f)
hc.printEncodings()

symbol with frequency 3 has encoding string [0, 0, 0]
symbol with frequency 20 has encoding string [0, 0, 1]
symbol with frequency 37 has encoding string [0, 1]
symbol with frequency 70 has encoding string [1]


In [14]:
import sys 

class ElementSet:
    def  __init__(self, i):
        self.setIndex = i 
        self.elementCount= 0
        self.elementDict = {}
    def addElement(self, e):
        if self.elementDict.get(e, None) is None:
            self.elementCount += 1
            self.elementDict[e] = 1
            e.addSet(self)
    def delElement(self, e):
      element = self.elementDict.get(e, None)
      if element is not None and element == 1:
          self.elementCount -= 1
          self.elementDict[e] = 0
                
    def getElementCount(self):
        return self.elementCount
    def printElementSet(self):
        print("[ ", end="")
        for e in self.elementDict.keys():
            if self.elementDict[e] == 1:
                print("{0} ".format(e.val), end="")
        print(" ]")
    def setCovered(self):
        #当集合被选中红，要把所包含元素覆盖掉
        for e in self.elementDict.keys():
            if self.elementDict[e] == 1:
                e.covered()
            
class Element:
    def __init__(self, val):
        self.val = val 
        self.sets = []
    def addSet(self, set):
        #把元素加入集合
        self.sets.append(set)
    def covered(self):
        #当元素被覆盖后，包含它的集合的元素个数要减1
        for set in self.sets:
            set.delElement(self)

In [15]:
eCount = 30
sCount = 6
eSet = []
sSet = []
elementNum = eCount

import random
#把eCount个元素随机分配到sCount个集合中

for i in range(sCount):
    s = ElementSet(i)
    sSet.append(s)
   
for i in range(1, eCount+1):
    e = Element(i)
    eSet.append(e)
   
    #随机的把元素分配到几个集合
    setGot = random.randint(1, sCount)
    while setGot > 0:
        setSelected = {}
        sel = random.randint(0, sCount-1)
        if setSelected.get(sel, None) is None:
            #记录下已经被选中的集合号
            setSelected[sel] = 1
            sSet[sel].addElement(e)
            setGot -= 1

In [16]:
def printAllSets():
    for s in sSet:
        s.printElementSet()
printAllSets()


while elementNum > 0:
    sel = 0;
    eCount = 0
    for i in range(len(sSet)):
        #选取当前含有未覆盖元素最大的集合
        if sSet[i].getElementCount() > eCount:
            eCount = sSet[i].getElementCount()
            sel = i 
    
    print("select set {0}, with element count {1}".format(sel, sSet[sel].getElementCount()))
    elementNum -= sSet[sel].getElementCount()        
    sSet[sel].setCovered()


[ 1 4 5 6 8 9 10 11 12 13 14 16 17 22 23 27 28 29 30  ]
[ 4 6 7 9 11 12 13 15 20 23 24 25 26  ]
[ 2 3 6 9 10 15 17 21 23 24 27 29 30  ]
[ 1 2 4 5 7 15 17 19 20 22 24 25 26 27  ]
[ 2 4 5 7 11 12 13 14 16 17 18 22 23 29 30  ]
[ 1 3 4 7 8 10 11 20 22 24 25 27 28 30  ]
select set 0, with element count 19
select set 3, with element count 8
select set 2, with element count 2
select set 4, with element count 1
